In [1]:
import json
import joblib
from pprint import pprint
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from imblearn.under_sampling import RandomUnderSampler

import lightgbm as lgb
from lightgbm.callback import log_evaluation, early_stopping

from isic_helper import DotDict
from isic_helper import compute_auc, compute_pauc

import matplotlib.pyplot as plt

%matplotlib inline
pd.options.display.max_columns = 1000

In [2]:
cfg = DotDict()
cfg.seed = 2022

cfg.models_output_dir = "models"
cfg.model_name = "lgb_v1"
cfg.fold_column = "gkf_fold"

cfg.sampling_ratio = 0.01

In [3]:
id_column = "isic_id"
target_column = "target"
group_column = "patient_id"

ord_categorical_features = [
    "sex",
    "tbp_lv_location",
    "tbp_tile_type",
    "tbp_lv_location_simple",
]

ohe_categorical_features = [
    "anatom_site_general", 
    "attribution",
]

attribution_mapper = {
    "Memorial Sloan Kettering Cancer Center": "MSKCC",
    "ACEMID MIA": "ACEMIDMIA",
    "Department of Dermatology, Hospital Clínic de Barcelona": "DoD_HCB",
    "University Hospital of Basel": "UHB",
    "Frazer Institute, The University of Queensland, Dermatology Research Centre": "FI_TUQ-DRC",
    "Department of Dermatology, University of Athens, Andreas Syggros Hospital of Skin and Venereal Diseases, Alexander Stratigos, Konstantinos Liopyris": "DoD_UA",
    "ViDIR Group, Department of Dermatology, Medical University of Vienna": "ViDIR"
}

def boosting_norm_feature(df, value_col, group_cols, err=1e-5):
    stats = ["mean", "std"]
    tmp = df.groupby(group_cols)[value_col].agg(stats)
    tmp.columns = [f"{value_col}_{stat}" for stat in stats]
    tmp.reset_index(inplace=True)
    df = df.merge(tmp, on=group_cols, how="left")
    feature_name = f"{value_col}_patient_norm"
    df[feature_name] = ((df[value_col] - df[f"{value_col}_mean"]) / 
                                       (df[f"{value_col}_std"] + err))
    return df, feature_name

def boosting_feature_engineering(df, err=1e-5):
    df["sex"] = df["sex"].fillna("missing_sex")
    df["anatom_site_general"] = df["anatom_site_general"].fillna("missing_anatom_site_general")
    df["tbp_tile_type"] = df["tbp_tile_type"].map({"3D: white": "white", "3D: XP": "XP"})
    df["attribution"] = df["attribution"].map(attribution_mapper)

    cols_to_norm = [
        "age_approx",
        "clin_size_long_diam_mm",
        "tbp_lv_A", "tbp_lv_Aext",
        "tbp_lv_B", "tbp_lv_Bext",
        "tbp_lv_C", "tbp_lv_Cext",
        "tbp_lv_H", "tbp_lv_Hext",
        "tbp_lv_L", "tbp_lv_Lext",
        "tbp_lv_areaMM2", "tbp_lv_area_perim_ratio",
        "tbp_lv_color_std_mean",
        "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", "tbp_lv_deltaLB", "tbp_lv_deltaLBnorm",
        "tbp_lv_eccentricity",
        "tbp_lv_minorAxisMM", "tbp_lv_nevi_confidence", "tbp_lv_norm_border",
        "tbp_lv_norm_color", "tbp_lv_perimeterMM",
        "tbp_lv_radial_color_std_max", "tbp_lv_stdL", "tbp_lv_stdLExt",
        "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle",
        "tbp_lv_x", "tbp_lv_y", "tbp_lv_z"
    ]
    numerical_features = cols_to_norm[:]
    for col in cols_to_norm:
        df, feature_name = boosting_norm_feature(df, col, ["patient_id"])
        numerical_features += [feature_name]
    
    df["tbp_lv_H_diff"] = df["tbp_lv_H"] - df["tbp_lv_Hext"]
    numerical_features += ["tbp_lv_H_diff"]

    df["tbp_lv_A_diff"] =  df["tbp_lv_Aext"] - df["tbp_lv_A"]
    df, feature_name = boosting_norm_feature(df, "tbp_lv_A_diff", ["patient_id", "tbp_lv_location"])
    numerical_features += [feature_name]

    df["tbp_lv_B_diff"] =  df["tbp_lv_Bext"] - df["tbp_lv_B"]
    df, feature_name = boosting_norm_feature(df, "tbp_lv_B_diff", ["patient_id", "tbp_lv_location"])
    numerical_features += [feature_name]
    
    df["num_images"] = df["patient_id"].map(df.groupby("patient_id")["isic_id"].count())
    numerical_features += ["num_images"]
    return df, numerical_features

In [4]:
INPUT_PATH = Path("/kaggle/input/isic-2024-challenge/")
MODELS_OUTPUT_PATH = Path(f"{cfg.models_output_dir}")
MODELS_OUTPUT_PATH.mkdir(exist_ok=True)

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False, na_values=["NA"])
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False, na_values=["NA"])

folds_df = pd.read_csv("/kaggle/input/isic-scd-folds/folds.csv")
train_metadata = train_metadata.merge(folds_df[[id_column, group_column, cfg.fold_column]], on=[id_column, group_column], how="left")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_metadata, numerical_features = boosting_feature_engineering(train_metadata)
test_metadata, _ = boosting_feature_engineering(test_metadata)

Train data size: (401059, 56)
Test data size: (3, 44)


In [5]:
mixed_encoded_preprocessor = ColumnTransformer(
    [
        ("numerical", "passthrough", numerical_features),
        (
            "ord_categorical",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-2, encoded_missing_value=-1, dtype=int),
            ord_categorical_features,
        ),
        (
            "ohe_categorical",
            OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown="ignore"),
            ohe_categorical_features
        )
    ],
    verbose_feature_names_out=False,

)
mixed_encoded_preprocessor.set_output(transform="pandas")

with open(f"{cfg.model_name}_encoder.joblib", "wb") as f:
    joblib.dump(mixed_encoded_preprocessor, f)

In [6]:
train_ids = train_metadata[id_column]
groups = train_metadata[group_column]
folds = train_metadata[cfg.fold_column]

enc = mixed_encoded_preprocessor.fit(train_metadata)
X_train = enc.transform(train_metadata)
y_train = train_metadata[target_column]

categorical_features = ord_categorical_features[:]
for ohe_col in ohe_categorical_features:
    categorical_features += [col for col in X_train.columns if col.startswith(ohe_col)]

print(f"Total number of columns: {len(X_train.columns)}")
X_train.head()

Total number of columns: 89


age_approx  clin_size_long_diam_mm   tbp_lv_A  tbp_lv_Aext   tbp_lv_B  \
0        60.0                    3.04  20.244422    16.261975  26.922447   
1        60.0                    1.10  31.712570    25.364740  26.331000   
2        60.0                    3.40  22.575830    17.128170  37.970460   
3        65.0                    3.22  14.242329    12.164757  21.448144   
4        55.0                    2.73  24.725520    20.057470  26.464900   

   tbp_lv_Bext   tbp_lv_C  tbp_lv_Cext   tbp_lv_H  tbp_lv_Hext   tbp_lv_L  \
0    23.954773  33.684638    28.953117  53.058545    55.828924  54.367448   
1    24.549290  41.219030    35.299260  39.702910    44.064040  48.861520   
2    33.485410  44.174920    37.611800  59.265850    62.909730  53.961180   
3    21.121356  25.746200    24.374023  56.414429    60.060388  18.649518   
4    25.710460  36.217980    32.608740  46.946070    52.041180  46.276310   

   tbp_lv_Lext  tbp_lv_areaMM2  tbp_lv_area_perim_ratio  \
0    62.025701        3.152561                27.476170   
1    55.362360        0.919497                12.235290   
2    61.670520        3.265153                24.184620   
3    23.314841        6.079940                14.889242   
4    54.855740        2.101708                19.902560   

   tbp_lv_color_std_mean  tbp_lv_deltaA  tbp_lv_deltaB  tbp_lv_deltaL  \
0                0.00000       3.982447       2.967674      -7.658253   
1                0.00000       6.347830       1.781713      -6.500838   
2                0.00000       5.447655       4.485044      -7.709336   
3                0.51452       2.077572       0.326788      -4.665323   
4                0.00000       4.668053       0.754434      -8.579431   

   tbp_lv_deltaLB  tbp_lv_deltaLBnorm  tbp_lv_eccentricity  \
0        8.360566            5.784302             0.901302   
1        6.839008            4.987244             0.639885   
2        9.092376            6.290359             0.932147   
3        4.783413            6.400196             0.654458   
4        9.148495            6.531302             0.946448   

   tbp_lv_minorAxisMM  tbp_lv_nevi_confidence  tbp_lv_norm_border  \
0            1.543016            2.628592e-03            7.091360   
1            0.821918            1.334303e-07            2.116402   
2            1.194905            2.959177e-04            4.798335   
3            2.481328            2.198945e+01            1.975874   
4            0.929916            1.378832e-03            3.658854   

   tbp_lv_norm_color  tbp_lv_perimeterMM  tbp_lv_radial_color_std_max  \
0           0.000000            9.307003                      0.00000   
1           0.000000            3.354148                      0.00000   
2           0.000000            8.886309                      0.00000   
3           1.771705            9.514499                      0.66469   
4           0.000000            6.467562                      0.00000   

   tbp_lv_stdL  tbp_lv_stdLExt  tbp_lv_symm_2axis  tbp_lv_symm_2axis_angle  \
0     2.036195        2.637780           0.590476                       85   
1     0.853227        3.912844           0.285714                       55   
2     1.743651        1.950777           0.361905                      105   
3     1.258541        1.573733           0.209581                      130   
4     2.085409        2.480509           0.313433                       20   

     tbp_lv_x     tbp_lv_y    tbp_lv_z  age_approx_patient_norm  \
0 -182.703552   613.493652  -42.427948                 0.000000   
1   -0.078308  1575.687000   57.174500                 0.000000   
2  123.649700  1472.010000  232.908900                 0.000000   
3 -141.024780  1442.185791   58.359802                 0.000000   
4  -72.315640  1488.720000   21.428960                 0.067495   

   clin_size_long_diam_mm_patient_norm  tbp_lv_A_patient_norm  \
0                            -0.463758              -0.105245   
1                            -1.843134               2.57208

In [7]:
def pauc_80(y_true, y_pred):
    score_value = compute_pauc(y_true, y_pred, min_tpr=0.8)   
    return "pauc_80", score_value, True


params = {
    "objective":        "binary",
    "verbosity":        -1,
    "boosting_type":    "gbdt",
    "metric":           "custom",
    "random_state":     cfg.seed,
    "lambda_l1":        0.08758718919397321, 
    "lambda_l2":        0.0039689175176025465, 
    "learning_rate":    0.01, 
    "max_depth":        4, 
    "num_leaves":       103, 
    "colsample_bytree": 0.8329551585827726, 
    "colsample_bynode": 0.4025961355653304, 
    "bagging_fraction": 0.7738954452473223, 
    "bagging_freq":     4, 
    "min_data_in_leaf": 85, 
    "scale_pos_weight": 2.7984184778875543,
}
num_rounds = 3000
es_rounds = 250
log_rounds = 50

In [8]:
best_num_rounds = {}
val_auc_scores = {}
val_pauc_scores = {}
all_folds = np.unique(folds)
oof_predictions = np.zeros(X_train.shape[0])
for fold in all_folds:
    print(f"Running fold: {fold}")
    dev_index = folds[folds != fold].index
    val_index = folds[folds == fold].index
    
    X_dev = X_train.loc[dev_index, :]
    y_dev = y_train[dev_index]

    rus = RandomUnderSampler(sampling_strategy=cfg.sampling_ratio, random_state=cfg.seed)
    X_dev, y_dev = rus.fit_resample(X_dev, y_dev)
    
    X_val = X_train.loc[val_index, :]
    y_val = y_train[val_index]
    
    model = lgb.LGBMClassifier(n_estimators=num_rounds, **params)
    model.fit(X_dev, y_dev,
              eval_set=[(X_val, y_val)], 
              eval_metric=pauc_80,
              callbacks=[
                    early_stopping(stopping_rounds=es_rounds, first_metric_only=True), 
                    log_evaluation(log_rounds)
                ]
             )
    best_num_rounds[f"fold_{fold}"] = model.best_iteration_
    
    val_preds = model.predict_proba(X_val)[:, -1]
    oof_predictions[val_index] = val_preds
    
    val_auc_scores[f"fold_{fold}"] = compute_auc(y_val, val_preds)
    val_pauc_scores[f"fold_{fold}"] = compute_pauc(y_val, val_preds, min_tpr=0.8)
    print("\n")
    
    with open(MODELS_OUTPUT_PATH / f"{cfg.model_name}_fold_{fold}.txt", "wb") as f:
        joblib.dump(model, f)

print("Val AUC scores:")
pprint(val_auc_scores)
print("Val PAUC scores:")
pprint(val_pauc_scores)

oof_train_preds_df = pd.DataFrame({
    id_column: train_ids,
    group_column: groups,
    "fold": folds,
    target_column: y_train,
    f"oof_{cfg.model_name}": oof_predictions
})
oof_train_preds_df.to_csv(f"oof_train_preds_{cfg.model_name}.csv", index=False)

cv_auc_oof = compute_auc(oof_train_preds_df[target_column], oof_train_preds_df[f"oof_{cfg.model_name}"])
cv_pauc_oof = compute_pauc(oof_train_preds_df[target_column], oof_train_preds_df[f"oof_{cfg.model_name}"], min_tpr=0.8)

cv_auc_avg = np.mean(list(val_auc_scores.values()))
cv_pauc_avg = np.mean(list(val_pauc_scores.values()))

cv_auc_std = np.std(list(val_auc_scores.values()))
cv_pauc_std = np.std(list(val_pauc_scores.values()))

print(f"CV AUC OOF: {cv_auc_oof}")
print(f"CV PAUC OOF: {cv_pauc_oof}")
print(f"CV AUC AVG: {cv_auc_avg}")
print(f"CV PAUC AVG: {cv_pauc_avg}")
print(f"CV AUC STD: {cv_auc_std}")
print(f"CV PAUC STD: {cv_pauc_std}")
print(f"CV PAUC RANGE: ({cv_pauc_avg - cv_pauc_std}, {cv_pauc_avg + cv_pauc_std})")
print(X_train.columns)

Running fold: 1
Training until validation scores don't improve for 250 rounds
[50]	valid_0's pauc_80: 0.173182
[100]	valid_0's pauc_80: 0.176999
[150]	valid_0's pauc_80: 0.178398
[200]	valid_0's pauc_80: 0.179118
[250]	valid_0's pauc_80: 0.180186
[300]	valid_0's pauc_80: 0.180431
[350]	valid_0's pauc_80: 0.181316
[400]	valid_0's pauc_80: 0.18193
[450]	valid_0's pauc_80: 0.182479
[500]	valid_0's pauc_80: 0.182844
[550]	valid_0's pauc_80: 0.182872
[600]	valid_0's pauc_80: 0.183002
[650]	valid_0's pauc_80: 0.18294
[700]	valid_0's pauc_80: 0.182689
[750]	valid_0's pauc_80: 0.182719
[800]	valid_0's pauc_80: 0.182898
[850]	valid_0's pauc_80: 0.182796
Early stopping, best iteration is:
[628]	valid_0's pauc_80: 0.183086
Evaluated only: pauc_80


Running fold: 2
Training until validation scores don't improve for 250 rounds
[50]	valid_0's pauc_80: 0.148115
[100]	valid_0's pauc_80: 0.151324
[150]	valid_0's pauc_80: 0.153825
[200]	valid_0's pauc_80: 0.15596
[250]	valid_0's pauc_80: 0.157025
[300]	

In [9]:
feature_importances = 0
for fold in all_folds:
    model_filepath = MODELS_OUTPUT_PATH / f"{cfg.model_name}_fold_{fold}.txt"
    with open(model_filepath, "rb") as f:
        model = joblib.load(f)
    if fold == 1:
        feature_names = model.feature_name_
    feature_importances += model.feature_importances_
feature_importances = feature_importances.astype(np.float32)
feature_importances /= len(all_folds)

In [10]:
imp = pd.DataFrame.from_dict(dict(zip(feature_names, feature_importances)), 
                             orient="index", columns=["imp"]).sort_values("imp", ascending=True)
imp.plot(kind="barh", figsize=(10, 30))
plt.show()

In [11]:
metadata = {
    "config": cfg.__dict__,
    "params": params,
    "num_rounds": num_rounds,
    "es_rounds": es_rounds,
    "best_num_rounds": best_num_rounds,
    "val_auc_scores": val_auc_scores,
    "val_pauc_scores": val_pauc_scores,
    "cv_auc_oof": cv_auc_oof,
    "cv_pauc_oof": cv_pauc_oof,
    "cv_auc_avg": cv_auc_avg,
    "cv_pauc_avg": cv_pauc_avg,
    "cv_auc_std": cv_auc_std,
    "cv_pauc_std": cv_pauc_std
}

with open(f"{cfg.model_name}_run_metadata.json", "w") as f:
    json.dump(metadata, f)

In [12]:
metadata

{'config': {'_temp': False,
  '_key': None,
  '_parent': None,
  'seed': 2022,
  'models_output_dir': 'models',
  'model_name': 'lgb_v1',
  'fold_column': 'gkf_fold',
  'sampling_ratio': 0.01},
 'params': {'objective': 'binary',
  'verbosity': -1,
  'boosting_type': 'gbdt',
  'metric': 'custom',
  'random_state': 2022,
  'lambda_l1': 0.08758718919397321,
  'lambda_l2': 0.0039689175176025465,
  'learning_rate': 0.01,
  'max_depth': 4,
  'num_leaves': 103,
  'colsample_bytree': 0.8329551585827726,
  'colsample_bynode': 0.4025961355653304,
  'bagging_fraction': 0.7738954452473223,
  'bagging_freq': 4,
  'min_data_in_leaf': 85,
  'scale_pos_weight': 2.7984184778875543},
 'num_rounds': 3000,
 'es_rounds': 250,
 'best_num_rounds': {'fold_1': 628,
  'fold_2': 1140,
  'fold_3': 1020,
  'fold_4': 476,
  'fold_5': 570},
 'val_auc_scores': {'fold_1': 0.9770033428143913,
  'fold_2': 0.9581817442962526,
  'fold_3': 0.966720744780422,
  'fold_4': 0.9701769602276273,
  'fold_5': 0.9719020013842372},
